In [43]:
from math import pi
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from datetime import date, datetime
from info_params import get_default_hparams
# from tf_utils.load_data import *
from pykalman import KalmanFilter

In [44]:
df = pd.read_csv("/home/nam/Dropbox/data/tac_CoinBase_BTC_1h.csv")

In [45]:
kf = KalmanFilter(transition_matrices = [1],
                  observation_matrices = [1],
                  initial_state_mean = df['Close'].values[0],
                  initial_state_covariance = 1,
                  observation_covariance=1,
                  transition_covariance=.01)

In [46]:
state_means,_ = kf.filter(df[['Close']].values)
state_means = state_means.flatten()

In [47]:
date_fmt = '%Y-%m-%d %I-%p'
df['date'] = df['Date'].apply(lambda x: datetime.strptime(x, hps.date_fmt))

In [48]:
mids = (df.Open + df.Close)/2
spans = abs(df.Close-df.Open)

inc = df.Close > df.Open
dec = df.Open > df.Close
w = 12*60*60*1000/24 # half day in ms
output_notebook()
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, toolbar_location="left",y_axis_label = "Price",
          x_axis_label = "Date")

p.segment(df.date, df.High, df.date, df.Low, color="black")
p.rect(df.date[inc], mids[inc], w, spans[inc], fill_color='green', line_color="green")
p.rect(df.date[dec], mids[dec], w, spans[dec], fill_color='red', line_color="red")
p.line(df.date,state_means,line_width=1,line_color = 'blue',legend="Kalman filter")

p.title.text = "Implementation of Kalman Filter Estimation - TCS EOD chart"
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3

Loading BokehJS ...

In [49]:
show(p)